In [63]:
import pandas as pd
import numpy as np
import gensim.downloader
import re
from gensim.models import word2vec, phrases
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_numeric,\
                    strip_non_alphanum, strip_multiple_whitespaces, strip_short
from textblob import TextBlob, Word
import collections

In [6]:
recipe_df = pd.read_csv('../../data/kaggle_recipes/RAW_recipes.csv')

In [7]:
recipe_df

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [9]:
#Extracting only Calories info from nutrition column
def nutrition_clean(i):
    i=i.strip("[]")
    i=i.split(", ")
    i = i[0]
    return i
    
recipe_df["calories_kcal"] = recipe_df["nutrition"].apply(nutrition_clean).astype("float64")

In [11]:
recipe_df.dtypes

name               object
id                  int64
minutes             int64
contributor_id      int64
submitted          object
tags               object
nutrition          object
n_steps             int64
steps              object
description        object
ingredients        object
n_ingredients       int64
calories_kcal     float64
dtype: object

In [15]:
#ingredients, and tags and steps are in string format, need to convert strings to list
def string_to_list(string):
    st = string.strip("[]").split(", ")
    s = [i.strip("''") for i in st]
    return s

In [18]:
recipe_df["tags_list"] = recipe_df["tags"].apply(string_to_list)

In [23]:
recipe_df.head(2)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories_kcal,tags_list
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,"[60-minutes-or-less, time-to-make, course, mai..."
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,"[30-minutes-or-less, time-to-make, course, mai..."


In [27]:
tag_list = []
for x in recipe_df["tags_list"]:
    for y in x:
        tag_list.append(y)

In [28]:
tag_list

['60-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'occasion',
 'north-american',
 'side-dishes',
 'vegetables',
 'mexican',
 'easy',
 'fall',
 'holiday-event',
 'vegetarian',
 'winter',
 'dietary',
 'christmas',
 'seasonal',
 'squash',
 '30-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'cuisine',
 'preparation',
 'occasion',
 'north-american',
 'breakfast',
 'main-dish',
 'pork',
 'american',
 'oven',
 'easy',
 'kid-friendly',
 'pizza',
 'dietary',
 'northeastern-united-states',
 'meat',
 'equipment',
 'time-to-make',
 'course',
 'preparation',
 'main-dish',
 'chili',
 'crock-pot-slow-cooker',
 'dietary',
 'equipment',
 '4-hours-or-less',
 '60-minutes-or-less',
 'time-to-make',
 'course',
 'main-ingredient',
 'preparation',
 'occasion',
 'side-dishes',
 'eggs-dairy',
 'potatoes',
 'vegetables',
 'oven',
 'easy',
 'dinner-party',
 'holiday-event',
 'easter',
 'cheese',
 'stove-top',
 'dietary',
 'christmas',
 'new-ye

In [29]:
tag_df = pd.DataFrame(tag_list ,columns = {"tags"})
tag_df["n_tags"] = 1
tag_df.groupby("tags").count().sort_values("n_tags", ascending = False)

,n_tags
tags,
preparation,230546
time-to-make,225326
course,218148
main-ingredient,170446
dietary,165091
...,...
marinara-sauce,1
middle-eastern-main-dish,1
breakfast-casseroles,1


In [37]:
recipe_df["ingredients_list"] = recipe_df["ingredients"].apply(string_to_list)

In [38]:
ingred_list =[]
for x in recipe_df["ingredients_list"]:
    for y in x:
        ingred_list.append(y)

In [39]:
ingred_df = pd.DataFrame(ingred_list, columns=["ingredients"])
ingred_df["n_ingred"] = 1
ingred_order = ingred_df.groupby("ingredients").sum().sort_values("n_ingred", ascending=False)
ingred_order

,n_ingred
ingredients,
salt,85746
butter,54975
sugar,44535
onion,39065
water,34914
...,...
fat free reduced-sugar cherry yogurt,1
fat free raspberry pecan salad dressing,1
reduced-fat beef hot dog,1


In [47]:
ingred_order[ingred_order['n_ingred']>1000]

,n_ingred
ingredients,
salt,85746
butter,54975
sugar,44535
onion,39065
water,34914
...,...
salmon fillets,1024
pork chops,1024
red bell peppers,1015


In [36]:
recipe_df["tags_list"] = recipe_df["tags"].apply(string_to_list)

In [41]:
recipe_df.dtypes

name                 object
id                    int64
minutes               int64
contributor_id        int64
submitted            object
tags                 object
nutrition            object
n_steps               int64
steps                object
description          object
ingredients          object
n_ingredients         int64
calories_kcal       float64
tags_list            object
ingredientss         object
ingredients_list     object
dtype: object

In [42]:
recipe_df.head(1)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories_kcal,tags_list,ingredientss,ingredients_list
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,"[60-minutes-or-less, time-to-make, course, mai...","[winter squash, mexican seasoning, mixed spice...","[winter squash, mexican seasoning, mixed spice..."


In [50]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:

model_wiki = gensim.downloader.load('glove-wiki-gigaword-50')
['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
model_wiki.most_similar('movie', topn=10)


In [64]:
total_ingredients = []
all_recipe_ingredients =  []

for i in range(len(recipe_df)):
    all_ingredients = list()
    
    #split each recipe into different ingredients
    ingred = recipe_df.loc[i, "ingredients"][1:-1]
    
    for ing in (ingred.split(',')):
        ing = remove_stopwords(ing)
        ing = strip_numeric(ing)
        ing = re.sub(r'\(.*oz.\)|(®)|(.*ed)|(.*ly)|boneless|skinless|chunks|fresh|large|cook drain|green|frozen|ground','',ing).strip()
        ing = strip_short(ing,2)
        ing = strip_multiple_whitespaces(ing)
        ing = strip_punctuation(ing)
        ing = strip_non_alphanum(ing)
        
        #convert plurals to singular e.g. tomatoes --> tomato
        ing = (" ".join(TextBlob(ing).words.singularize()))
        
        all_ingredients.append(ing)
        total_ingredients.append(ing)
        
    all_recipe_ingredients.append(all_ingredients)
    
counts_ingr = collections.Counter(total_ingredients)

print('Total Ingredients (with repetition):  \t{}'.format((len(total_ingredients))))
print('Unique Ingredients : \t\t\t{}'.format((len(counts_ingr.values()))))
print('Total Receipes:  \t\t\t{}'.format((len(all_recipe_ingredients))))


Total Ingredients (with repetition):  	2103719
Unique Ingredients : 			10952


NameError: name 'all_receipes_ingredients' is not defined

In [68]:
context = 10          # Context window size; 

downsampling = 1e-2   # threshold for configuring which 
                      # higher-frequency words are randomly downsampled

model = word2vec.Word2Vec(all_recipe_ingredients, \
            vector_size=300, min_count=4, \
            window=context, sample = downsampling)

In [73]:
similar_words = {search_term: [item[0] for item in model.wv.most_similar([search_term], topn=5)]
                  for search_term in ['egg','mango','bread', 'rice']}


In [71]:
similar_words

{'egg': ['extra egg',
  'egg substitute',
  'jumbo egg',
  'egg beater egg substitute',
  'egg white'],
 'mango': ['papaya', 'nectarine', 'cantaloupe', 'kiwi fruit', 'navel orange'],
 'bread': ['white bread',
  'whole wheat bread',
  'sourdough bread',
  'stale bread',
  'french bread'],
 'rice': ['white rice',
  'long grain rice',
  'long grain white rice',
  'raw rice',
  'brown rice']}